NHS Digital have published their [methodology for high risk shielded patient list identification](https://digital.nhs.uk/coronavirus/shielded-patient-list/methodology/medicines-data). They provided the list as BNF codes and following notebook generates codes compliant with the NHS Dictionary of Medicines and Devices which is the local UK Snomed drug extension. We will use this rule to identify severe asthma

In [7]:
from ebmdatalab import bq
import os
import pandas as pd

### Inclusion Criteria
NHS Digital inclusion criteria rule 1 states "Patients with asthma were identified as being prescribed Long acting beta2-agonist (LABA) as either a LABA or in combination with an inhaled corticosteroid (LABA/ICS) OR prescriptions for a leukotriene receptor antagonist (e.g. monteluekast).""

In [8]:
## The following is written based on version 1 frm March 27th and 
## is archived at https://web.archive.org/save/https://digital.nhs.uk/coronavirus/shielded-patient-list/methodology/medicines-data

sql = '''
WITH bnf_codes AS (  
  SELECT bnf_code FROM hscic.presentation WHERE 
  (bnf_code LIKE '030302%' OR #BNF leukotriene antagonists
  bnf_code LIKE '0301011B0%' OR #BNF bambuterol
  bnf_code LIKE '0301011E0%' OR #BNF formoterol
  bnf_code LIKE '0301011U0%' OR #BNF salmeterol
  bnf_code LIKE '0302000C0%' OR #BNF Beclometasone dipropionate
  bnf_code LIKE '0302000K0%' OR #BNF budesonide
  bnf_code LIKE '0302000V0%' OR #BNF Fluticasone furoate 
  bnf_code LIKE '0302000N0%') #BNF Fluticasone propionate 
   )
   
SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

UNION ALL

SELECT "amp" AS type, id, bnf_code, descr
FROM dmd.amp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

ORDER BY type, bnf_code, id'''

severe_asthma_inc_rule1 = bq.cached_read(sql, csv_path=os.path.join('..','data','severe_asthma_inc_rule1.csv'))
pd.set_option('display.max_rows', None)
severe_asthma_inc_rule1.head(10)

,type,id,bnf_code,nm
0,amp,375711000001100,0301011B0BBAAAA,Bambec 10mg tablets (AstraZeneca UK Ltd)
1,amp,809411000001102,0301011B0BBABAB,Bambec 20mg tablets (AstraZeneca UK Ltd)
2,amp,3206811000001109,0301011E0BBAAAA,Foradil 12microgram inhalation powder capsules...
3,amp,3243511000001107,0301011E0BCAAAB,Oxis 6 Turbohaler (AstraZeneca UK Ltd)
4,amp,3245011000001103,0301011E0BCABAC,Oxis 12 Turbohaler (AstraZeneca UK Ltd)
5,amp,9628711000001106,0301011E0BDAAAD,Atimos Modulite 12micrograms/dose inhaler (Chi...
6,amp,11176411000001106,0301011E0BEAAAE,Formoterol Easyhaler 12micrograms/dose dry pow...
7,amp,3084011000001103,0301011U0BBAAAA,Serevent 50microgram disks with Diskhaler (Gla...
8,amp,3082411000001100,0301011U0BBABAB,Serevent 50microgram disks (GlaxoSmithKline UK...
9,amp,408911000001105,0301011U0BBACAC,Serevent 25micrograms/dose inhaler (GlaxoSmith...


NHS Digital inclusion criteria rule 2 states "From the above list of patients, those who had been dispensed 4 or more prescriptions for prednisolone between July 2019 and December 2019 were identified and considered to have severe asthma. "

We may wish to consider a different criteria for dispensed. e.g. we could do must have had at least 60 x 5mg prenisolone tabs in last three months

In [9]:
sql = '''
WITH bnf_codes AS (  
  SELECT bnf_code FROM hscic.presentation WHERE 
  bnf_code LIKE '0603020T0%' #BNF oral prednisolone
   )
SELECT "vmp" AS type, id, bnf_code, nm
FROM dmd.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

UNION ALL

SELECT "amp" AS type, id, bnf_code, descr
FROM dmd.amp
WHERE bnf_code IN (SELECT * FROM bnf_codes)

ORDER BY type, bnf_code, id'''

severe_asthma_inc_rule2 = bq.cached_read(sql, csv_path=os.path.join('..','data','severe_asthma_inc_rule2.csv'))
pd.set_option('display.max_rows', None)
severe_asthma_inc_rule2.head(10)


,type,id,bnf_code,nm
0,amp,52911000001107,0603020T0AAAAAA,Prednisolone 1mg tablets (Alliance Healthcare ...
1,amp,238511000001107,0603020T0AAAAAA,Prednisolone 1mg tablets (Teva UK Ltd)
2,amp,646211000001106,0603020T0AAAAAA,Prednisolone 1mg tablets (Accord Healthcare Ltd)
3,amp,649711000001109,0603020T0AAAAAA,Prednisolone 1mg tablets (A A H Pharmaceutical...
4,amp,707911000001100,0603020T0AAAAAA,Prednisolone 1mg tablets (Kent Pharmaceuticals...
5,amp,798611000001108,0603020T0AAAAAA,Prednisolone 1mg tablets (The Boots Company Plc)
6,amp,844111000001103,0603020T0AAAAAA,Prednisolone 1mg tablets (Wockhardt UK Ltd)
7,amp,9807711000001108,0603020T0AAAAAA,Prednisolone 1mg tablets (Almus Pharmaceutical...
8,amp,10409611000001103,0603020T0AAAAAA,Prednisolone 1mg tablets (Arrow Generics Ltd)
9,amp,14786411000001103,0603020T0AAAAAA,Prednisolone 1mg tablets (Boston Healthcare Ltd)


**Severe Asthma**  = <br/> 
Presence of at least one from `severe_asthma_inc_rule1` in last three months <br/>
AND <br/>
X issues of at least one from `severe_asthma_inc_rule2` in the last three months